In [ ]:
# Notebook to prototype calculating the bounding box of all the features in a geodata frame.
# 

import pandas as pd
import geopandas as gp

In [ ]:
%matplotlib notebook

In [ ]:
# Directory in which user's output CSV report data was saved - it will now be our *input* directory
my_sandbox_dir = r'S:/my_modx_output_dir/'

In [ ]:
# Name of CSV file with volume, V/C, and speed data for selected links - it will now be our *input* CSV file
csv_fn = 'links_report_base_scenario.csv'

In [ ]:
# Fully-qualified pathname to CSV file
fq_csv_fn = my_sandbox_dir + csv_fn

In [ ]:
links_data_df = pd.read_csv(fq_csv_fn, delimiter=',')

In [ ]:
links_data_df

In [ ]:
# List of the IDs for the model network links for which data is reported in the input CSV file
links_list = links_data_df['ID1'].to_list()

In [ ]:
# Directory in which the spatial data for the model network links is stored (both shapefile and GeoJSON formats)
links_spatial_data_dir = r'G:/Data_Resources/modx/statewide_links_shapefile/'

In [ ]:
# Load the links shapefile into a geopandas dataframe 
# NOTE: This version of the shapefile is in EPSG4326, i.e., WGS84
links_shapefile_fn = 'Statewide_Links_2018_BK_EPSG4326.shp'
fq_links_shapefile_fn = links_spatial_data_dir + links_shapefile_fn
links_gdf = gp.read_file(fq_links_shapefile_fn)
links_gdf.set_index("ID")

In [ ]:
# Filter the links geodataframe to only the links of interest
filtered_links_gdf = links_gdf[links_gdf['ID'].isin(links_list)] 

In [ ]:
filtered_links_gdf

In [ ]:
# shorthand
gdf = filtered_links_gdf

In [ ]:
bounds_tuples = gdf['geometry'].map(lambda x: x.bounds)

In [ ]:
bounds_dicts = []

In [ ]:
for t in bounds_tuples:
    temp = { 'minx' : t[0], 'miny' : t[1], 'maxx' : t[2], 'maxy' : t[3] }
    bounds_dicts.append(temp)


In [ ]:
bounds_dicts[0]

In [ ]:
boundsdf = pd.DataFrame(bounds_dicts)

In [ ]:
boundsdf

In [ ]:
minx = boundsdf['minx'].min()

In [ ]:
miny = boundsdf['miny'].min()

In [ ]:
maxx = boundsdf['maxx'].max()

In [ ]:
maxy = boundsdf['maxy'].max()

In [ ]:
# Return the bounding box of all the features in a geo-dataframe.
# The bounding box is returned as a dictionary with the following keys: { 'minx', 'miny', 'maxx', 'maxy'}.
#
def bbox_of_gdf(gdf):
    bounds_tuples = gdf['geometry'].map(lambda x: x.bounds)
    bounds_dicts = []
    for t in bounds_tuples:
        temp = { 'minx' : t[0], 'miny' : t[1], 'maxx' : t[2], 'maxy' : t[3] }
        bounds_dicts.append(temp)
    # end_for
    bounds_df = pd.DataFrame(bounds_dicts)
    minx = bounds_df['minx'].min()
    miny = bounds_df['miny'].min()
    maxx = bounds_df['maxx'].max()
    maxy = bounds_df['maxy'].max()
    retval = { 'minx' : minx, 'miny' : miny, 'maxx' : maxx, 'maxy' : maxy }
    return retval
# end_def bbox_of_gdf()

In [ ]:
my_bounds = bbox_of_gdf(gdf)

In [ ]:
my_bounds

In [ ]:
center_x = minx + (maxx - minx) / 2

In [ ]:
center_y = miny + (maxy - miny) / 2

In [ ]:
center_x, center_y

In [ ]:
# Given a dictonary of the form  'minx', 'miny', 'maxx', 'maxy'} representing a geographic bounding box,
# return the center point as a dictionary with the keys { 'x' , 'y' }.
def center_of_bbox(bbox):
    center_x = bbox['minx'] + (bbox['maxx'] - bbox['minx']) / 2
    center_y = bbox['miny'] + (bbox['maxy'] - bbox['miny']) / 2
    retval = { 'x' : center_x, 'y' : center_y }
    return retval
# end_def center_of_bbox()

In [ ]:
center_point = center_of_bbox(my_bounds)

In [ ]:
center_point